In [1]:
# Transformers installation
! pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00


In [2]:
# Parameters
model_checkpoint_name = "buddhilive/bert-base-zero"
my_model_name = "bert-nopretrained-squad"
num_train_epochs = 3
max_length = 384
stride = 50

In [3]:
# Login to Huggingface
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# Load SQuAD dataset
from datasets import load_dataset

squad = load_dataset("squad", split="train[:10000]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
# Train and test split
squad = squad.train_test_split(test_size=0.2)

squad["train"][0]

{'id': '56cf5c5faab44d1400b89129',
 'title': 'Spectre_(2015_film)',
 'context': "In the United States and Canada, the film opened on 6 November 2015, and in its opening weekend, was originally projected to gross $70–75 million from 3,927 screens, the widest release for a Bond film. However, after grossing $5.25 million from its early Thursday night showings and $28 million on its opening day, weekend projections were increased to $75–80 million. The film ended up grossing $70.4 million in its opening weekend (about $20 million less than Skyfall's $90.6 million debut, including IMAX previews), but nevertheless finished first at the box office. IMAX generated $9.1 million for Spectre at 374 screens, premium large format made $8 million from 429 cinemas, reaping 11% of the film's opening, which means that Spectre earned $17.1 million (23%) of its opening weekend total in large-format venues. Cinemark XD generated $1.85 million in 112 XD locations.",
 'question': 'What day of the week was 

In [6]:
# Tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# check if it is fast Tokenizer
tokenizer.is_fast

True

In [7]:
# Preprocess dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
# Map and Tokenize Data
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
# create a data collector
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [10]:
# Set optimizers
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [11]:
# Import pretrained model
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint_name)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at buddhilive/bert-base-zero and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# validation preprocess
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [13]:
# Initialize dataset
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

'''
tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
'''

'\ntf_validation_set = model.prepare_tf_dataset(\n    tokenized_squad["test"],\n    shuffle=False,\n    batch_size=16,\n    collate_fn=data_collator,\n)\n'

In [14]:
validation_dataset = squad["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["test"].column_names,
)
len(squad["test"]), len(validation_dataset)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

(2000, 2021)

In [15]:
tf_validation_set = model.prepare_tf_dataset(
    validation_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [27]:
# compile model
import tensorflow as tf

model.compile(optimizer=optimizer)

model.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  124055040 
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
Total params: 124056578 (473.24 MB)
Trainable params: 124056578 (473.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Huggingface Push callback
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=my_model_name,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/buddhilive/bert-nopretrained-squad into local empty directory.


Download file tf_model.h5:   0%|          | 8.00k/473M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/473M [00:00<?, ?B/s]

In [18]:
import tensorflow as tf
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [19]:
# Setup timer
import time
train_time = time.time()
print(train_time)

1694346951.8908052


In [28]:
# model training
train_time = time.time()
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=20, callbacks=[callback])
train_time = time.time() - train_time

Epoch 1/20
500/500 [==============================] - 721s 1s/step - loss: 3.8040 - val_loss: 0.0000e+00
Epoch 2/20
500/500 [==============================] - 714s 1s/step - loss: 3.8048 - val_loss: 0.0000e+00
Epoch 3/20
500/500 [==============================] - 720s 1s/step - loss: 3.8013 - val_loss: 0.0000e+00
Epoch 4/20
500/500 [==============================] - 716s 1s/step - loss: 3.8023 - val_loss: 0.0000e+00
Epoch 5/20
500/500 [==============================] - 714s 1s/step - loss: 3.8023 - val_loss: 0.0000e+00
Epoch 6/20
500/500 [==============================] - 710s 1s/step - loss: 3.8059 - val_loss: 0.0000e+00
Epoch 7/20
500/500 [==============================] - 716s 1s/step - loss: 3.8000 - val_loss: 0.0000e+00
Epoch 8/20
500/500 [==============================] - 715s 1s/step - loss: 3.8018 - val_loss: 0.0000e+00
Epoch 9/20
500/500 [==============================] - 712s 1s/step - loss: 3.7983 - val_loss: 0.0000e+00
Epoch 10/20
500/500 [==============================] - 

In [29]:
#predict
predictions = model.predict(tf_validation_set)

127/127 [==============================] - 63s 467ms/step


In [30]:
import evaluate

metric = evaluate.load("squad")

In [31]:
# evaluate Function
from tqdm.auto import tqdm
import collections
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [32]:
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    validation_dataset,
    squad["test"],
)

  0%|          | 0/2000 [00:00<?, ?it/s]

{'exact_match': 3.25, 'f1': 9.472855166333263}

In [33]:
# Inference
question = "what is my name?"
context = "I'm from Sri lanka. My name is Buddhi."

from transformers import pipeline

question_answerer = pipeline("question-answering", model=my_model_name)
question_answerer(question=question, context=context)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-nopretrained-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


{'score': 0.10935772210359573,
 'start': 20,
 'end': 37,
 'answer': 'My name is Buddhi'}

In [34]:
minutes, seconds = divmod(train_time, 60)
hours, minutes = divmod(minutes, 60)
print("%d:%02d:%02d" % (hours, minutes, seconds))

4:01:38
